In [ ]:
import sys

In [ ]:
sys.path.append("../Traffic4cast")

In [ ]:
sys.path

In [ ]:
import os
import argparse

import statistics
from collections import defaultdict

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import torch
import torch.nn.functional as F
import torch_geometric
import tqdm
from torch import nn
from pathlib import Path
import numpy as np
import random
from sklearn.model_selection import KFold

import t4c22
from t4c22.metric.masked_crossentropy import get_weights_from_class_fractions
from t4c22.misc.t4c22_logging import t4c_apply_basic_logging_config
from t4c22.t4c22_config import class_fractions
from t4c22.t4c22_config import load_basedir
from t4c22.dataloading.t4c22_dataset_geometric import T4c22GeometricDataset

t4c_apply_basic_logging_config(loglevel="DEBUG")

BASEDIR = load_basedir(fn="t4c22_config.json", pkg=t4c22)

In [ ]:
torch.__version__

In [ ]:
torch.cuda.is_available()

In [ ]:
HIDDEN_CHANNELS = 32
NUM_LAYERS = 3
NUM_EDGE_CLASSES = 3
NUM_FEATURES = 4
DROPOUT = 0.0
BATCH_SIZE = 2
CITY = "london"
nan_to_num_map = {"london": -1.21, "melbourne": -0.8, "madrid": -0.56}
SEED = 1234
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.cuda.manual_seed(SEED)
device = torch.device("cuda", 0)

In [ ]:
BASEDIR

In [ ]:
class RecLinear(nn.Module):
    def __init__(self, num_edges, num_nodes, num_attrs, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):

        super(RecLinear, self).__init__()

        self.embed = nn.Embedding(num_edges, hidden_channels)
        self.node_embed = nn.Embedding(num_nodes, hidden_channels)
        self.node_embed1 = nn.Embedding(num_nodes, 4)
        self.time_embed = nn.Embedding(96, hidden_channels)
        self.week_embed = nn.Embedding(7, hidden_channels)
        self.node_index = torch.arange(0, num_nodes).to(device)

        self.node_lin = nn.Linear(in_channels, hidden_channels)
        self.node_lin1 = nn.Linear(hidden_channels * 2, hidden_channels)
        self.attr_lin = nn.Linear(num_attrs, hidden_channels)
        self.attr_lin1 = nn.Sequential(nn.Linear(num_attrs, hidden_channels), nn.LeakyReLU(),
                                       nn.Linear(hidden_channels, hidden_channels))
        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(hidden_channels * 6, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.fc1 = nn.Linear(num_nodes, 256)
        self.fc2 = nn.Linear(256, 32)  # mean vector
        self.fc3 = nn.Linear(256, 32)  # standard deviation vector
        self.fc4 = nn.Linear(32, 256)
        self.fc5 = nn.Linear(256, num_nodes)

        from torch_geometric.nn import GATv2Conv as GCNConv

        self.conv1 = torch.nn.ModuleList()
        for i in range(3):
            self.conv1.append(GCNConv(hidden_channels, hidden_channels, edge_dim=hidden_channels))

        self.conv2 = torch.nn.ModuleList()
        for i in range(3):
            self.conv2.append(GCNConv(hidden_channels, hidden_channels, edge_dim=hidden_channels))

        self.gcn_lin1 = nn.Linear(hidden_channels * 2, hidden_channels)
        self.gcn_lin2 = nn.Linear(hidden_channels * 2, hidden_channels)

    def gelu(self, x):
        return 0.5 * x * (1 + F.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * torch.pow(x, 3))))

    # coding process
    def encode(self, x):
        h = self.gelu(self.fc1(x))
        return self.fc2(h), self.fc3(h)

    # Gaussian sampling
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    # decoding process
    def decode(self, z):
        h = self.gelu(self.fc4(z))
        h = self.fc5(h)
        return h

    def reset_parameters(self):
        self.embed.reset_parameters()
        self.node_embed.reset_parameters()
        self.node_embed1.reset_parameters()
        self.time_embed.reset_parameters()
        self.node_lin.reset_parameters()
        self.node_lin1.reset_parameters()
        self.attr_lin.reset_parameters()
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        self.fc3.reset_parameters()
        self.fc4.reset_parameters()
        self.fc5.reset_parameters()
        for lin in self.lins:
            lin.reset_parameters()
        for lin in self.conv1:
            lin.reset_parameters()
        for lin in self.conv2:
            lin.reset_parameters()
        self.gcn_lin1.reset_parameters()
        self.gcn_lin2.reset_parameters()

    def forward(self, index, edge_index, x, attr, cur_t, cur_w):
        mask_idx = (torch.sum(x, dim=1, keepdim=True) != nan_to_num_map[CITY] * 4).type(torch.float)

        xmax = 23.91
        xmin = nan_to_num_map[CITY]
        x_norm = (x - xmin) / (xmax - xmin)

        ratio = 0.8 + 0.4 * np.random.rand(1)[0]

        x_norm = x_norm * ratio

        drop_idx = (torch.rand_like(x_norm[:, 0:1]) > 0.4).type(torch.float)
        x_norm = x_norm * drop_idx

        x_norm = torch.transpose(x_norm, 0, 1)
        mu, log_var = self.encode(x_norm)
        z = self.reparameterize(mu, log_var)
        x_rec = self.decode(z)

        x_rec = x_rec / ratio

        x_rec = torch.transpose(x_rec, 0, 1)
        x_rec = x_rec * (xmax - xmin) + xmin
        x_rec1 = mask_idx * x + (1 - mask_idx) * x_rec

        attr1 = self.attr_lin(attr)
        embed = self.embed(index)

        node_embed = self.node_embed(self.node_index)
        pre_data = node_embed
        for conv in self.conv1:
            node_embed = conv(node_embed, edge_index, attr1)
            node_embed = self.gelu(node_embed) + pre_data

        data = self.gelu(self.node_lin(x_rec1))
        pre_data = data
        for conv in self.conv2:
            data = conv(data, edge_index, attr1)
            data = self.gelu(data) + pre_data

        x_i = torch.index_select(data, 0, edge_index[0])
        x_j = torch.index_select(data, 0, edge_index[1])
        x = torch.concat([x_i, x_j], dim=1)
        x = self.gcn_lin1(x)

        x_i = torch.index_select(node_embed, 0, edge_index[0])
        x_j = torch.index_select(node_embed, 0, edge_index[1])
        x1 = torch.concat([x_i, x_j], dim=1)
        x1 = self.gcn_lin2(x1)

        time_embed = self.time_embed(cur_t.long())
        week_embed = self.week_embed(cur_w.long())

        xf = torch.cat([embed, self.attr_lin1(attr), x, x1, time_embed, week_embed], dim=1)

        for lin in self.lins[:-1]:
            xf = lin(xf)
            xf = self.gelu(xf)

        xf = self.lins[-1](xf)

        return xf, x_rec

In [ ]:
dataset = T4c22GeometricDataset(root=BASEDIR, city=CITY,
    edge_attributes=["speed_kph", "parsed_maxspeed", "length_meters", "counter_distance",
        'num_lanes', 'mean_pop', 'catering', 'leisure', 'shopping', 'tourism', 'traffic', 'transport',
        'pois_total', 'greeneries', 'water_bodies', 'built_up', 'other', "importance", "highway", "oneway",],
    split="train", fill=-1, normalize="zs", cachedir=Path(f"{BASEDIR}/cache"), idx=0)

In [ ]:
dataset.__dict__.keys()

In [ ]:
dataset[0]

In [ ]:
len(dataset)

In [ ]:
dataset[-1]

In [ ]:
dataset.edge_attr.shape

In [ ]:
test_dataset = T4c22GeometricDataset(root=BASEDIR, city=CITY,
                                     edge_attributes=["speed_kph", "parsed_maxspeed", "length_meters",
                                                      "counter_distance", "importance", "highway", "oneway", ],
                                     split="test", fill=-1,
                                     normalize="sz", cachedir=Path(f"{BASEDIR}/cache"), idx=0)

In [ ]:
print("Dataset Size\t", len(dataset))
print("Test Dataset Size\t", len(test_dataset))
print("The statistics of training set are: Min [%d]\tMax [%d]\tMean [%.4f]\tStd[%.4f]" % (
    dataset.min_volume, dataset.max_volume, dataset.mean_volume, dataset.std))

In [ ]:
spl = int(((0.8 * len(dataset)) // 2) * 2)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [spl, len(dataset) - spl])
print("Train Dataset Size\t", len(train_dataset))
print("Validation Dataset Size\t", len(val_dataset))

In [ ]:
train_dataset.dataset

In [ ]:
next(iter(train_dataset))

In [ ]:
next(iter(train_dataset))

In [ ]:
city_class_fractions = class_fractions[CITY]
city_class_weights = torch.tensor(
    get_weights_from_class_fractions(
        [city_class_fractions['green'], city_class_fractions['yellow'],
         city_class_fractions['red']])).float()
print("City Class Weight\t", city_class_weights)

In [ ]:
city_class_weights = city_class_weights.to(device)
edge_index = dataset.edge_index.to(device)
edge_attr = dataset.edge_attr.to(device)

In [ ]:
num_edges = edge_index.shape[1]
num_attrs = edge_attr.shape[1]
num_nodes = np.max(edge_index.cpu().numpy()) + 1
print('num_nodes', num_nodes, "num_edges", num_edges, "num_attrs", num_attrs)

In [ ]:
index = torch.arange(0, num_edges).to(device)

In [ ]:
model = RecLinear(num_edges, num_nodes, num_attrs, NUM_FEATURES, HIDDEN_CHANNELS,
                  NUM_EDGE_CLASSES, NUM_LAYERS, DROPOUT).to(device)

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)

In [ ]:
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(fold, len(train_ids), len(test_ids))
    print(train_ids[:5])

In [ ]:
train_ids, test_ids = next(iter(kfold.split(dataset)))

In [ ]:
len(dataset)

In [ ]:
len(train_ids), len(test_ids)

In [ ]:
model.reset_parameters()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
loss_f = torch.nn.CrossEntropyLoss(weight=city_class_weights, ignore_index=-1)
loss_mse = torch.nn.MSELoss()

min_loss = 10000

# print("fold", fold)
train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

In [ ]:
train_subsampler

In [ ]:
model.train()

In [ ]:
losses = []
optimizer.zero_grad()

In [ ]:
pbar = tqdm.tqdm(
    torch_geometric.loader.dataloader.DataLoader(dataset, batch_size=BATCH_SIZE,
                                                 num_workers=2, sampler=train_subsampler),
    "train",
    total=len(train_dataset) // BATCH_SIZE, )

In [ ]:
count = 0
for data in pbar:
    data = data.to(device)
    data.x[data.x > 23.91] = 23.91
    data.x[data.x == -1] = nan_to_num_map[CITY]
    loss = 0.
    print(data)
    break

In [ ]:
data

In [ ]:
if (count == 0):
    lens = data.x.shape[0] // BATCH_SIZE
    lens1 = data.y.shape[0] // BATCH_SIZE
    count += 1

In [ ]:
lens, lens1

In [ ]:
data.y.shape[0] // lens1

In [ ]:
edge_index[0]

In [ ]:
data.t[0]

In [ ]:
data.t[1]

In [ ]:
data.week[0], data.week[1]

In [ ]:
i = 0
data.y[i * lens1:(i + 1) * lens1]

In [ ]:
data.x[i * lens:(i + 1) * lens]

In [ ]:
for i in range(data.y.shape[0] // lens1):
    t = data.t[i]
    cur_t = torch.ones_like(edge_index[0]) * t
    week = data.week[i]
    cur_week = torch.ones_like(edge_index[0]) * week

    y = data.y[i * lens1:(i + 1) * lens1].nan_to_num(-1)
    x = data.x[i * lens:(i + 1) * lens]
    y_hat, x_rec = model(index, edge_index, x,
                         edge_attr, cur_t, cur_week)
    y = y.long()

    train_index = torch.nonzero(torch.sum(x, dim=1) != nan_to_num_map[CITY] * 4).squeeze()

    rec_loss = loss_mse(x[train_index], x_rec[train_index])
    acc_loss = loss_f(y_hat, y)

    loss += rec_loss + acc_loss

In [ ]:
edge_index

In [ ]:
x

In [ ]:
edge_attr

In [ ]:
edge_attr.shape

In [ ]:
cur_t

In [ ]:
cur_week

In [ ]:
week

In [ ]:
mask_idx = (torch.sum(x, dim=1, keepdim=True) != nan_to_num_map[CITY] * 4).type(torch.float)

In [ ]:
torch.sum(x, dim=1, keepdim=True).shape

In [ ]:
xmax = 23.91
xmin = nan_to_num_map[CITY]
x_norm = (x - xmin) / (xmax - xmin)

In [ ]:
x_norm

In [ ]:
ratio = 0.8 + 0.4 * np.random.rand(1)[0]
ratio

In [ ]:
x_norm = x_norm * ratio

In [ ]:
torch.rand_like(x_norm[:, 0:1])

In [ ]:
drop_idx = (torch.rand_like(x_norm[:, 0:1]) > 0.4).type(torch.float)
drop_idx

In [ ]:
x_norm = x_norm * drop_idx

In [ ]:
x_norm.shape

In [ ]:
x_norm = torch.transpose(x_norm, 0, 1)
x_norm.shape

In [ ]:
mu, log_var = model.encode(x_norm)

In [ ]:
z = model.reparameterize(mu, log_var)

In [ ]:
x_rec = model.decode(z)

In [ ]:
x_rec

In [ ]:
x_rec = x_rec / ratio

In [ ]:
x_rec = torch.transpose(x_rec, 0, 1)
x_rec.shape

In [ ]:
x_rec = x_rec * (xmax - xmin) + xmin

In [ ]:
x_rec

In [ ]:
x_rec1 = mask_idx * x + (1 - mask_idx) * x_rec

In [ ]:
x_rec1

In [ ]:
y

In [ ]:
y.unique()

In [ ]:
x

# Testing inference mode

In [ ]:
model.eval()

In [ ]:
idx = 0
data = next(iter(test_dataset))
data

In [ ]:
data = data.to(device)

In [ ]:
data.x

In [ ]:
data.x[data.x > 23.91] = 23.91
data.x

In [ ]:
data.x[data.x == -1] = nan_to_num_map[CITY]
data.x

In [ ]:
t = data.t
t

In [ ]:
cur_t = torch.ones_like(edge_index[0]) * t
cur_t

In [ ]:
week = data.week
week

In [ ]:
cur_week = torch.ones_like(edge_index[0]) * week

In [ ]:
save_path = "save/cc/class_L3_H32_F-1_Nzs_B2_e20/london_2/"
y_pred = 0.
for fold in range(5):
    for mm in range(16, 21):
        print(f"{save_path}model_{fold}_{mm:03d}.pt")
        model.load_state_dict(torch.load(f"{save_path}model_{fold}_{mm:03d}.pt", map_location='cuda:0'))
        model.eval()
        for ii in range(5):
            y_hat, _ = model(index, edge_index, data.x, edge_attr, cur_t, cur_week)
            y_hat = y_hat.detach()
            print(y_hat.shape, y_hat[:5])
            y_pred += y_hat
        break
y_pred /= 125.

In [ ]:
y_pred

In [ ]:
df = test_dataset.torch_road_graph_mapping._torch_to_df_cc(data=y_pred, day="test", t=idx)

In [ ]:
df.head()

In [ ]:
dfs = []
for idx, data in tqdm.tqdm(enumerate(test_dataset), total=len(test_dataset)):
    data = data.to(device)
    data.x[data.x > 23.91] = 23.91
    data.x[data.x == -1] = nan_to_num_map[opt['city']]

    t = data.t
    cur_t = torch.ones_like(edge_index[0]) * t
    week = data.week
    cur_week = torch.ones_like(edge_index[0]) * week

    if (opt['city'] == 'melbourne'):
        y_pred = 0.
        for fold in range(5):
            model.load_state_dict(torch.load(f"{opt['save_path']}model_best_{fold}.pt", map_location='cuda:0'))
            model.eval()
            for ii in range(5):
                y_hat, _ = model(index, edge_index, data.x, edge_attr, cur_t, cur_week)
                y_hat = y_hat.detach()
                y_pred += y_hat
        y_pred /= 25.
    else:
        y_pred = 0.
        for fold in range(5):
            for mm in range(16, 21):
                model.load_state_dict(torch.load(f"{opt['save_path']}model_{fold}_{mm:03d}.pt", map_location='cuda:0'))
                model.eval()
                for ii in range(5):
                    y_hat, _ = model(index, edge_index, data.x, edge_attr, cur_t, cur_week)
                    y_hat = y_hat.detach()
                    y_pred += y_hat
        y_pred /= 125.

    df = test_dataset.torch_road_graph_mapping._torch_to_df_cc(data=y_pred, day="test", t=idx)
    dfs.append(df)
df = pd.concat(dfs)
df["test_idx"] = df["t"]
del df["day"]
del df["t"]

submission = df
# print(submission.head(20))

# (BASEDIR / "submissions" / opt['submission_name'] / opt['city'] / "labels").mkdir(exist_ok=True, parents=True)
# table = pa.Table.from_pandas(submission)
# pq.write_table(table, BASEDIR / "submissions" / opt['submission_name'] / opt[
#     'city'] / "labels" / f"cc_labels_test.parquet", compression="snappy")


In [ ]:
df